In [1]:
#!/usr/local/bin/python3.6

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from scapy.all import *

packets = rdpcap('./pcap/test.pcapng')

Using TensorFlow backend.


In [16]:
#traitement des paquets ###############################################

i = 0
for packet in packets:
    for cat in packet:
        print(cat.decode("hex"))
    '''
    tab[i][0] = packet[1].proto()
    tab[i][1] = packet[1].src()
    tab[i][2] = packet[1].dst()
    tab[i][3] = packet[2].sport()
    tab[i][4] = packet[2].dport()
    tab[i][4] = packet[1].flags()
    tab[i][0] = packet[1].options()
    tab[i][0] = packet[1].load()
    i += 1
    '''

AttributeError: decode

In [ ]:
for i in range(20):
    #afficher 20 paquets et leur menace 

In [ ]:
# reshape éventuel ###############################################

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
#mettre dans une variable la menace 0 ou 1

In [ ]:
# création du modèle 

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

In [ ]:
# compilation du model ###############################################

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# training  ###############################################

model.fit(X_train, Y_train, batch_size=128, nb_epoch=4, verbose=1,
         validation_data=(X_test, Y_test))

In [ ]:
  # evualtion ###############################################
         
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# résultat ###############################################

# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]